# 👕 코디 크롤링 노트북

## 크롤링 목록
- 코디 고유 ID
- 스타일
- 코디 URL
- 이미지 URL
- 태그
- 코디에 포함된 item들의 ID

## Excel View Extension 설치해서 확인 가능!

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from easydict import EasyDict
from tqdm import tqdm
from typing import Tuple

# pip install openpyxl
import openpyxl
import time
import os

In [24]:
# 🚀 코디들을 뽑아올 페이지 지정
COORDI_LIST_PATH = 'https://www.musinsa.com/app/styles/lists?page='
COORDI_BASE_PATH = "https://www.musinsa.com/app/styles/views/"
NUM_CRAWL_PAGE = 5

In [25]:
# 🚀 Chrome option 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(argument='--headless')
chrome_options.add_argument(argument='--no-sandbox')
chrome_options.add_argument(argument='--disable-dev-shm-usage')

In [26]:
# 🚀 크롤링 완료된 정보를 저장할 excel sheet_codi 지정
wb_codi = openpyxl.Workbook()
sheet_codi = wb_codi.active
sheet_codi.append(["id",  "style", "img_url", "url"])

wb_codi_tag = openpyxl.Workbook()
sheet_codi_tag = wb_codi_tag.active
sheet_codi_tag.append(["id", "tag"])

wb_codi_item_id = openpyxl.Workbook()
sheet_codi_item_id = wb_codi_item_id.active
sheet_codi_item_id.append(["id", "item_id"])

In [27]:
# 🚀 (코디 링크, 스타일 정보, 코디 이미지 url) 리스트 받아오기
def get_codi_info(driver: webdriver.Chrome) -> Tuple[list, list, list]:
    
    codi_link_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-item__thumbnail > a")
    style_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-information__text")
    image_url_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-item__thumbnail > a > div.style-list-thumbnail > img")

    print (f"""
           # of codi_links : {len(codi_link_result)}
           # of style : {len(style_result)}
           # of img_urls : {len(image_url_result)}
           """)

    codi_id_list = list()
    codi_style_list = list()
    codi_url_list = list()

    # ⭐ 1. 코디 ID 정보 javascript function 인자로 부터 받아오기
    for link_element in codi_link_result:
        js_function = link_element.get_attribute("onclick")
        codi_id = js_function.split("'")[1]
        codi_id_list.append(codi_id)
        
    # ⭐ 2. style 정보 text로 추출하기
    for style_element in style_result:
        codi_style_list.append(style_element.text)
        
    # ⭐ 3. 코디의 이미지 url 받아오기
    for img_element in image_url_result:
        codi_url_list.append(img_element.get_attribute('src'))
        
    return codi_id_list, codi_style_list, codi_url_list

In [28]:
# 🚀 코디 링크에 하나씩 접속하면서, 연관된 상품 ID, 코디태그 받아오기
def make_crawl_xlsx(driver: webdriver.Chrome) -> None:
    
    # (코디 링크, 스타일 정보, 코디 이미지 url) 리스트 받아오기
    codi_id_list, codi_style_list, codi_url_list = get_codi_info(driver)
    
    for codi_id, codi_style, codi_img_url in tqdm(zip(codi_id_list, codi_style_list, codi_url_list), total=len(codi_id_list), desc="Codi crawling progress"):
            
        # 하나의 코디에 대한 information 저장
        codi_info = list()
        
        # ⭐ 4. 코디의 상세정보가 있는 url 받아오기
        codi_path = COORDI_BASE_PATH + codi_id
        
        # 코디의 경로 받아오고 코디 상세정보 페이지 진입하기
        driver.get(codi_path)
        driver.implicitly_wait(3)
        
        # ⭐ 5. 코디 태그 받아오기 (다른 sheet에 저장)
        coordi_tags = driver.find_elements(by=By.CSS_SELECTOR, value=".ui-tag-list__item")
        for tag_element in coordi_tags:
            sheet_codi_tag.append([codi_id, tag_element.text])
        
        # ⭐ 6. 현재 코디에 포함된 아이템 id 받아오기 (다른 sheet에 저장)
        item_elements = driver.find_elements(by=By.CSS_SELECTOR, value=".styling_img")
        for item_element in item_elements:
            item_id = item_element.get_attribute("href").split("/")[-2]
            sheet_codi_item_id.append([codi_id, item_id])
            
        codi_info.append(codi_id)
        codi_info.append(codi_style)
        codi_info.append(codi_img_url)
        codi_info.append(codi_path)
        sheet_codi.append(codi_info)

In [29]:
# 🚀 크롤링 완료된 파일 저장
def save_as_xlsx() -> None:
    os.makedirs('./asset', exist_ok=True)
    wb_codi.save("./asset/codi.xlsx")
    wb_codi_tag.save("./asset/codi_tag.xslx")
    wb_codi_item_id.save("./asset/codi_item_id.xlsx")

In [30]:
# 🚀 최상위 메인 페이지 불러오기 (코디 목록 60개 보여지는 페이지)
def do_crawling(num_crawl_pages: int = 5) -> None:
    driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)
    
    for page_idx in range(1, num_crawl_pages + 1):
        print (f"Crawling {page_idx} pages..")
        driver.get(COORDI_LIST_PATH + str(page_idx))
        time.sleep(3)
        make_crawl_xlsx(driver)
    
    driver.close()
    
    save_as_xlsx()
    
    print ("Crawling done. All files saved (./asset)")

In [ ]:
# 🚀 지정한 페이지 수 만큼 크롤링 진행하기
do_crawling(NUM_CRAWL_PAGE)